In [1]:
!pip install gensim


In [2]:
from gensim import corpora
from gensim.models import LdaModel

In [3]:
!pip install spacy
!python -m spacy download it_core_news_lg
# !python -m spacy download en_core_web_sm


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 996.6 kB/s eta 0:00:00
✔ Downl

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='it')

In [11]:
# Fase di preprocessing e tokenizzazione

import spacy
import pandas as pd

nlp = spacy.load("it_core_news_lg")
# spell = SpellChecker(language='it')

def preprocess(text):
   # print(text)
  doc = nlp(text)

  return [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and token.pos_ in {"NOUN", "PROPN"} ] # is_alpha?? '''and token.pos_ in {"NOUN", "VERB"}'''


In [28]:
reviews = pd.read_csv('product_reviews_comments.csv', delimiter='|', header=None)
reviews


,0,1,2,3,4
0,B0CLML6FM8,Utile e ben fatto,E' uno dei resoconti piu' ordinati che abbia m...,9 persone l'hanno trovato utile,9
1,B0CLML6FM8,attuale,Bel libro attuale e snocciola le problematiche...,Una persona l'ha trovato utile,1
2,B0CLML6FM8,israele e palestinesi,"interessante ma troppo denso di informazioni ,...",NaN,0
3,B0CLML6FM8,Molto articolato,Il Giornalista indipendente per antonomasia è ...,6 persone l'hanno trovato utile,6
4,B0CLML6FM8,Buono per una visione a volo d'uccello,Fornisce una ossatura cronologica degli eventi...,Una persona l'ha trovato utile,1
...,...,...,...,...,...
3743,8807036525,Leggetelo.,"Parte bene da subito, questo bellissimo raccon...",Una persona l'ha trovato utile,1
3744,8807036525,Ni,"Carino, ma nulla di eccezionale.",NaN,0
3745,8807036525,Un'occasione mancata,"Troppo sbrigativo, tutto troppo rapido. Si sen...",2 persone l'hanno trovato utile,2
3746,8807036525,Fantastico romanzo d'esordio!,"Già nel 1993, quando cominciò a farsi notare a...",Una persona l'ha trovato utile,1


In [29]:
from gensim.corpora import Dictionary

texts = []

for idx, row in reviews.iterrows():
  text_clean = (row[1] + " " + ' ' if pd.isna(row[2]) else row[2]).replace('.', ' ')
  texts.append(text_clean) # TODO FORSE DOVREI AGGIUNGERE IL FATTO DI 'SPEZZARE'/'PULIRE IL TESTO DA POSSIBILI PAROLE LEGATE COL PUNTO

# Preprocess texts with SpaCy
processed_texts = [preprocess(text) for text in texts]
print(processed_texts)

[['resoconte', 'prefazione', 'espressione', 'autore', 'stile', 'dettaglio', 'evento', 'avviso', 'appendice', 'cronistoria', 'punto', 'chiave', 'uso', 'stile', 'pensiero', 'fatto', 'metafora', 'inversione', 'davide', 'golia', 'punto', 'rottura', 'schieramento', 'palestinese', 'bello', 'lampo'], ['libro', 'problematica', 'conflitto', 'israeliano', 'palestinese', 'dinamica', 'approfondimento', 'passaggio'], ['informazione', 'avvenimento'], ['giornalista', 'antonomasia', 'garanzia', 'semplicità', 'linguaggio', 'testo', 'filo', 'protagonista', 'fatto', 'storia', 'termine', 'lettura', 'sensazione', 'premessa', 'vicenda', 'terra', 'ragione', 'torto', 'torto', 'ragione', 'ragione', 'ragione'], ['ossatura', 'evento', 'tema', 'punto', 'vista', 'travaglio', 'persona'], ['conto', 'situazione', 'israele', 'palestina', 'libro', 'colpa', 'opinione'], ['marco', 'travaglio', 'testo', 'sintassi', 'filo', 'discorso'], ['travaglio', 'stile', 'storia', 'palestina', 'conflitto', 'libro', 'idea', 'medio', 'o

In [30]:
# Bisogna togliere dal testo le 5 parole più comuni

print(processed_texts)
from collections import Counter

# Flatten the list of lists into one list of all tokens
all_lemmas = [lemma for doc in processed_texts for lemma in doc]

# Count total term frequencies
term_freq = Counter(all_lemmas)

print(term_freq)
print(term_freq.most_common(15))
print(len(term_freq))

for doc_text in processed_texts:
  #print(doc_text)
  for t in doc_text:
    for common_term in term_freq.most_common(5):
      #print(common_term)
      if doc_text.count(common_term[0]) > 0:
        doc_text.remove(common_term[0])

[['resoconte', 'prefazione', 'espressione', 'autore', 'stile', 'dettaglio', 'evento', 'avviso', 'appendice', 'cronistoria', 'punto', 'chiave', 'uso', 'stile', 'pensiero', 'fatto', 'metafora', 'inversione', 'davide', 'golia', 'punto', 'rottura', 'schieramento', 'palestinese', 'bello', 'lampo'], ['libro', 'problematica', 'conflitto', 'israeliano', 'palestinese', 'dinamica', 'approfondimento', 'passaggio'], ['informazione', 'avvenimento'], ['giornalista', 'antonomasia', 'garanzia', 'semplicità', 'linguaggio', 'testo', 'filo', 'protagonista', 'fatto', 'storia', 'termine', 'lettura', 'sensazione', 'premessa', 'vicenda', 'terra', 'ragione', 'torto', 'torto', 'ragione', 'ragione', 'ragione'], ['ossatura', 'evento', 'tema', 'punto', 'vista', 'travaglio', 'persona'], ['conto', 'situazione', 'israele', 'palestina', 'libro', 'colpa', 'opinione'], ['marco', 'travaglio', 'testo', 'sintassi', 'filo', 'discorso'], ['travaglio', 'stile', 'storia', 'palestina', 'conflitto', 'libro', 'idea', 'medio', 'o

In [31]:
dictionary = Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1)], [(14, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)], [(29, 1), (30, 1)], [(9, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 4), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1)], [(8, 1), (17, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1)], [(32, 1), (48, 1), (56, 1), (57, 1), (58, 1)], [(21, 1), (24, 1), (48, 1), (54, 1), (59, 1), (60, 1), (61, 1)], [(62, 1), (63, 1), (64, 1)], [(65, 1)], [(47, 1)], [(66, 1), (67, 1)], [], [(55, 1), (68, 1), (69, 1)], [(21, 1), (35, 1), (66, 1), (70, 1), (71, 1)], [(72, 1)], [(2, 1)], [(34, 1), (48, 1)], [(51, 1), (70, 1), (73, 1), (74, 1)], [(17, 1), (21, 1), (24, 1), (48, 1), (53, 1), (60, 1), (61, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1),

In [40]:
import random

random.seed(42)
# Creazione e allenamento del modello LDA e visualizzazione dei topic risultanti
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    passes=100,
    random_state=42
)

# Display topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")
    with open('topics_list' + '.txt', 'a', newline='') as file:
        file.write(f"Topic {idx}: {topic}" + "\n")

Topic 0: 0.010*"famiglia" + 0.010*"idea" + 0.008*"psicologia" + 0.008*"destra" + 0.007*"società" + 0.007*"capitolo" + 0.007*"pensiero" + 0.006*"senso" + 0.006*"figlio" + 0.006*"d"
Topic 1: 0.020*"contenuto" + 0.013*"racconto" + 0.013*"genere" + 0.011*"verità" + 0.010*"acquisto" + 0.010*"stella" + 0.008*"insegnamento" + 0.008*"berlinguer" + 0.008*"libricino" + 0.007*"pensiero"
Topic 2: 0.022*"travaglio" + 0.018*"situazione" + 0.018*"fatto" + 0.015*"guerra" + 0.013*"fabbri" + 0.012*"israele" + 0.012*"evento" + 0.010*"conflitto" + 0.009*"realtà" + 0.009*"geopolitica"
Topic 3: 0.021*"pensiero" + 0.017*"vannacci" + 0.014*"cosa" + 0.012*"problema" + 0.012*"opinione" + 0.011*"argomento" + 0.011*"concetto" + 0.011*"persona" + 0.010*"capitolo" + 0.010*"società"
Topic 4: 0.020*"punto" + 0.014*"vista" + 0.014*"processo" + 0.011*"eichmann" + 0.009*"stile" + 0.007*"argomento" + 0.007*"riflessione" + 0.007*"fatto" + 0.007*"scuola" + 0.007*"politica"
Topic 5: 0.009*"palestinese" + 0.009*"israele" + 0

In [41]:
# Verifica dei topic per ciascun documento
for idx, doc in enumerate(corpus):
    print(doc)
    topic_distribution = lda_model.get_document_topics(doc, minimum_probability=0.0)
    most_probable_topic = max(topic_distribution, key=lambda x: x[1])[0]
    print(f"{idx + 1}|{most_probable_topic}")
    print(topic_distribution)
    print(lda_model.print_topic(most_probable_topic))
    csv_row=''
    if (most_probable_topic == 0):
        print(f"{idx + 1}|{most_probable_topic}|Società e identità")
        csv_row = f"{idx + 1}|{most_probable_topic}|Società e identità"
    elif (most_probable_topic == 1):
      print(f"{idx + 1}|{most_probable_topic}|Esperienza di lettura")
      csv_row=f"{idx + 1}|{most_probable_topic}|Esperienza di lettura"
    elif (most_probable_topic == 2):
      print(f"{idx + 1}|{most_probable_topic}|Geopolitica")
      csv_row=f"{idx + 1}|{most_probable_topic}|Geopolitica"
    elif (most_probable_topic == 3):
      print(f"{idx + 1}|{most_probable_topic}|Vannacci")
      csv_row=f"{idx + 1}|{most_probable_topic}|Vannacci"
    elif (most_probable_topic == 4):
      print(f"{idx + 1}|{most_probable_topic}|Giudizio morale e memoria")
      csv_row=f"{idx + 1}|{most_probable_topic}|Giudizio morale e memoria"
    elif (most_probable_topic == 5):
      print(f"{idx + 1}|{most_probable_topic}|Conflitto Israelo-Palestinese")
      csv_row=f"{idx + 1}|{most_probable_topic}|Conflitto Israelo-Palestinese"
    elif (most_probable_topic == 6):
      print(f"{idx + 1}|{most_probable_topic}|Qualità del libro")
      csv_row=f"{idx + 1}|{most_probable_topic}|Qualità del libro"
    elif (most_probable_topic == 7):
      print(f"{idx + 1}|{most_probable_topic}|Analisi ideologica del fascismo")
      csv_row=f"{idx + 1}|{most_probable_topic}|Analisi ideologica del fascismo"
    elif (most_probable_topic == 8):
      print(f"{idx + 1}|{most_probable_topic}|Esperienza d’acquisto")
      csv_row=f"{idx + 1}|{most_probable_topic}|Esperienza d’acquisto"
    elif (most_probable_topic == 9):
      print(f"{idx + 1}|{most_probable_topic}|Storia e memoria del fascismo in Italia")
      csv_row=f"{idx + 1}|{most_probable_topic}|Storia e memoria del fascismo in Italia"

    with open('product_reviews_comments_topics_20250527' + '.csv', 'a', newline='') as file:
        file.write(csv_row + "\n")



Output streaming troncato alle ultime 5000 righe.
[(82, 1), (3922, 1), (5303, 1)]
2749|7
[(0, 0.02500729), (1, 0.025007626), (2, 0.025008501), (3, 0.025008094), (4, 0.025007185), (5, 0.025007185), (6, 0.025016135), (7, 0.7749175), (8, 0.025012288), (9, 0.025008217)]
0.027*"fascismo" + 0.021*"analisi" + 0.021*"eco" + 0.018*"orsini" + 0.013*"viaggio" + 0.010*"pagina" + 0.010*"informazione" + 0.008*"prof" + 0.008*"mente" + 0.008*"passato"
2749|7|Analisi ideologica del fascismo
[(607, 1), (770, 1), (1265, 1), (3371, 1), (5293, 1), (5304, 1)]
2750|0
[(0, 0.8713969), (1, 0.0142881945), (2, 0.014288676), (3, 0.014292237), (4, 0.01428873), (5, 0.014288102), (6, 0.01428994), (7, 0.014288278), (8, 0.014289985), (9, 0.014288967)]
0.010*"famiglia" + 0.010*"idea" + 0.008*"psicologia" + 0.008*"destra" + 0.007*"società" + 0.007*"capitolo" + 0.007*"pensiero" + 0.006*"senso" + 0.006*"figlio" + 0.006*"d"
2750|0|Società e identità
[(16, 1), (66, 1), (163, 1), (231, 1), (371, 1), (390, 1), (564, 2), (2855

In [ ]:
# Verifiche e controlli dei lemma/token/testi per verificarne la pulizia e il possibile filtraggio

print(processed_texts)
from collections import Counter

# Flatten the list of lists into one list of all tokens
all_lemmas = [lemma for doc in processed_texts for lemma in doc]

# Count total term frequencies
term_freq = Counter(all_lemmas)
print(term_freq)

# Voglio verificare le prime 5 parole più utilizzate
print(term_freq.most_common(5))
print(len(term_freq))

# Step 2: Count frequencies
total = sum(term_freq.values())
print(total)

# Step 3: Set frequency thresholds (e.g. 0.5% and 95%)
min_freq = total * 0.0025
max_freq = total * 0.95
print(min_freq)
print(max_freq)

# Step 4: Create filtered vocab
filtered_vocab = {lemma for lemma, freq in term_freq.items() if min_freq < freq < max_freq}
print(filtered_vocab)
# Step 5: Filter each document
# filtered_docs = [
#     [token.lemma_.lower() for token in doc if token.lemma_.lower() in filtered_vocab]
#     for doc in docs
# ]

In [ ]:
# Tentativo per recuperare le affinità semantiche delle parole presenti nei topic del modello LDA.
# Il risultato non è stato dei migliori, e si è optato per l'assegnazione manuale.

print(lda_model.print_topic(0))
lda_model.get_topic_terms(0)
print(lda_model.show_topic(0))


token1 = nlp("israele")[0]
token2 = nlp("palestina")[0]

print(token1.similarity(token2))

for topic in lda_model.show_topics():
  print(topic)
  print(topic[0])
  print(topic[1])
  for term_topic in lda_model.show_topic(topic[0]):
    print(term_topic)
    for term_topic2 in lda_model.show_topic(0):
      token1 = nlp(term_topic[0])[0]
      token2 = nlp(term_topic2[0])[0]
      if token1.similarity(token2) > 0.5 and term_topic[0] != term_topic2[0]:
        print('COPPIA UTILE')
        print('token similarity ' + term_topic[0] + ' ' + term_topic2[0] +  str(token1.similarity(token2)))



In [ ]:
# Before pruning
dictionary = Dictionary(processed_texts)

print(dictionary.get('comandante'))
print(dictionary.get('cinese'))

word='cinese'
if word in dictionary.token2id:
    word_id = dictionary.token2id[word]
    freq = dictionary.dfs[word_id]  # number of documents that contain the word
    print(f"The word '{word}' appears in {freq} document(s).")
else:
    print(f"The word '{word}' is not in the dictionary.")

full_dictionary = Dictionary(processed_texts)
all_tokens = set(full_dictionary.token2id.keys())

# After pruning
pruned_dictionary = Dictionary(processed_texts)
pruned_dictionary.filter_extremes(no_below=2, no_above=0.95)
kept_tokens = set(pruned_dictionary.token2id.keys())

# Pruned tokens = those not kept
removed_tokens = all_tokens - kept_tokens

print("Removed tokens:", removed_tokens)

print(dictionary)


for most_common_terms in term_freq.most_common(5):
  print(most_common_terms)
  print(most_common_terms[0])
  print(most_common_terms[1])
  print(dictionary.get(most_common_terms[0]))
  dictionary.filter_tokens(most_common_terms[0])
  print(dictionary.get(most_common_terms[0]))



print(pruned_dictionary)
corpus = [dictionary.doc2bow(text) for text in processed_texts]
print(corpus)

None
None
The word 'cinese' appears in 3 document(s).
Removed tokens: {'divinità', 'approfondire', 'work', 'vegetarianesimo', 'accredito', 'interferenza', 'veramente', 'pauca', 'eficaz', 'lazarillo', 'gropius', 'enconomia', 'occidentemi', 'marocchino', 'turkmenistan', 'suicidio', 'burlo', 'sadismo', 'lore', 'brutalità', 'inciampo', 'brodo', 'stupendo', 'decreto', 'cuco', 'rarità', 'sostanziale', 'mobile', 'segezia', 'autodefirre si', 'ispettore', 'espediento', 'addis', 'pacifisti', 'predica', 'guterrez', 'carne', 'difficoltá', 'albo', 'lgtbq', 'burocrata', 'avveninente', 'doetro', 'black', 'delude', 'dipendenza', 'elia', 'estabilishment', 'scontato', 'mediocri', 'corea', 'omofobismo', 'carl', 'scorrect', 'nader', 'fiammante', 'nebbia', 'lealtà', 'menefreghismo', 'plastificata', 'mussolinia', 'nane', 'orda', 'asse', 'bancarotta', 'mediocre', 'falce', 'fiumealmare', 'sfregio', 'disegnino', 'diversitá', 'stufa', 'tt', 'quanro', 'inquietante', 'nuova', 'bolsonaro', 'dispetto', 'aguzzino', 

In [ ]:
dictionary = Dictionary(processed_texts)
print(dictionary.dfs)
freqs = sorted(dictionary.dfs.items(), key=lambda x: -x[1])
top5_ids = [token_id for token_id, _ in freqs[:10]]
# remove them
for tid in top5_ids:
  print(tid)
  print(dictionary.get(tid))
  dictionary.filter_tokens([tid])
dictionary.compactify()  # re-reindex to close gaps
corpus = [dictionary.doc2bow(text) for text in processed_texts]

{17: 4, 15: 21, 7: 18, 20: 84, 6: 38, 8: 66, 1: 31, 0: 4, 4: 7, 3: 29, 21: 38, 9: 136, 13: 1, 11: 3, 5: 1, 10: 1, 16: 20, 18: 3, 19: 4, 14: 24, 2: 28, 12: 4, 27: 10, 23: 69, 25: 6, 24: 63, 22: 30, 26: 35, 29: 96, 28: 20, 33: 49, 30: 1, 32: 21, 38: 13, 34: 58, 31: 27, 36: 28, 40: 47, 39: 25, 35: 14, 43: 43, 41: 30, 37: 50, 42: 6, 44: 1, 46: 100, 48: 112, 47: 71, 45: 73, 50: 48, 54: 97, 51: 56, 53: 34, 49: 24, 52: 93, 56: 14, 57: 4, 55: 47, 58: 129, 59: 11, 60: 15, 61: 17, 63: 3, 62: 8, 64: 47, 66: 3, 65: 150, 67: 108, 68: 76, 69: 21, 70: 2, 71: 4, 72: 70, 73: 22, 80: 64, 75: 37, 77: 1, 74: 22, 79: 19, 78: 5, 76: 153, 81: 1, 82: 21, 99: 79, 101: 3, 89: 2, 83: 1, 95: 9, 84: 24, 87: 40, 88: 35, 91: 15, 98: 33, 104: 1, 103: 1, 93: 7, 85: 13, 90: 11, 102: 18, 96: 17, 92: 17, 105: 4, 86: 48, 97: 10, 100: 7, 94: 52, 117: 125, 137: 28, 114: 64, 129: 96, 136: 7, 127: 5, 118: 10, 131: 1, 130: 68, 124: 46, 110: 18, 107: 133, 128: 20, 109: 49, 126: 8, 112: 28, 122: 42, 111: 55, 108: 3, 135: 33, 134